In [9]:
# Anaísa Forti da Fonseca
# 11811ECP012

import numpy as np
import matplotlib.pyplot as plt
import control as ctl
from IPython.display import display
from ipywidgets import interact, fixed

def graph_size(n):
    """Função auxiliar para definir tamanho dos gráficos"""
    return (n*(1+5**0.5)/2, n)

In [10]:
R1, R2, L, C = 10, 300, 10e-1, 10e-6
V_i = 10  # Vcc

@interact(R1=fixed(R1), R2=fixed(R2), L=fixed(L), C=fixed(C), eq=False, mf=True)
def simula_RLC_serie(R1, R2, L, C, eq, mf=True):
    # Considerando a malha de realimentação
    circuito_aberto = ctl.tf([R2],[C*L*R2, (C*R1*R2)+L, R1])
    circuito_fechado = ctl.feedback(circuito_aberto)

    # Considerando a FT equivalente
    circuito_equivalente = ctl.tf([1/(L*C)],[1, (C*R1*R2+L)/(C*L*R2), (R1+R2)/(L*C*R2)])

    print("Zeros:", circuito_equivalente.zero())
    print("Polos:", circuito_equivalente.pole())

    t = np.linspace(0, 0.1, 1000)
    to, vo = ctl.step_response(circuito_fechado, t)
    vo = vo*V_i
    to2, vo2 = ctl.step_response(circuito_equivalente, t)
    vo2 = vo2*V_i
    step = lambda t : np.array(list(1 if tau > 0 else 0 for tau in t))

    fig = plt.figure(figsize=graph_size(7))
    plt.plot(t, V_i*step(t), label='entrada $v_i(t)$')
    if mf:
        plt.plot(to, vo, label='saída $v_o(t)$, mf')
    if eq:
        plt.plot(to2, vo2, label='saída $v_o(t)$, eq')
    plt.ylabel("amplitude [V]")
    plt.xlabel("tempo [s]")
    plt.legend()
    plt.show()

interactive(children=(Checkbox(value=False, description='eq'), Checkbox(value=True, description='mf'), Output(…